# Imputation Research Project <img src="https://chroniclesofai.com/content/images/2021/05/file-20201210-18-elk4m.jpg" alt="Alt text image not displaying" width="500" align="right" />
## Notebook 3.0: Autoencoder Model

**Author:** Chike Odenigbo

**Date:** November 25th, 2022

**Notebook Structure:**

* 1.0 Preprocessing

* **1.1 Exploratory Data Analysis**

* 1.2 Masking

* 2.* Models


Water Sugar lutein_zeaxanthin Alcohol

In [1]:
import pandas as pd
import os
from src.visualization.visualize import histogram, box_plot, bar_plot
from pathlib import Path
from notebook_config import ROOT_DIR  # setup.py file changed the root of the project so it is set in the config file

ROOT_DIR = ROOT_DIR.as_posix()  # convert root path to windows readable path (i.e. change backslash to forward slash)
from joblib import load
import tensorflow as tf
from keras import backend as K
import keras
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt

In [2]:
notebook_nm = "3.0-autoencoder"
fig_dir = f"{ROOT_DIR}/reports/figures/"
output_prefix = notebook_nm

In [3]:
pd.set_option('display.max_columns', None)

# Ground Truth Included
water_df = pd.read_csv(f'{ROOT_DIR}/data/processed/water.csv')
sugars_df = pd.read_csv(f'{ROOT_DIR}/data/processed/sugars.csv')
lutein_df = pd.read_csv(f'{ROOT_DIR}/data/processed/lutein.csv')

# Scaled Data without Ground Truth to prevent Data Leakage
water_mcar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/water_mcar_scaled.csv', index_col = 0)
water_mar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/water_mar_scaled.csv', index_col = 0)

lutein_mar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/lutein_mar_scaled.csv', index_col = 0)
lutein_mcar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/utein_mcar_scaled.csv', index_col = 0)

sugars_mar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/sugars_mar_scaled.csv', index_col = 0)
sugars_mcar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/sugars_mcar_scaled.csv', index_col = 0)

water_mcar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/water_mcar_scaled.csv', index_col = 0)
water_mar_scaled_df = pd.read_csv(f'{ROOT_DIR}/data/processed/water_mar_scaled.csv', index_col = 0)

# Scalers to return back to origin scale for model evaluation
scaler_lutein_mar = load(f'{ROOT_DIR}/models/scaler_lutein_mar.joblib')
scaler_lutein_mcar = load(f'{ROOT_DIR}/models/scaler_lutein_mcar.joblib')

scaler_sugars_mar = load(f'{ROOT_DIR}/models/scaler_sugars_mar.joblib')
scaler_sugars_mcar = load(f'{ROOT_DIR}/models/scaler_sugars_mcar.joblib')

scaler_water_mcar = load(f'{ROOT_DIR}/models/scaler_water_mcar.joblib')
scaler_water_mar = load(f'{ROOT_DIR}/models/scaler_water_mar.joblib')

In [51]:
(scaler_water_mcar.feature_names_in_)

array(['serving_size', 'total_fat', 'saturated_fat', 'cholesterol',
       'sodium', 'choline', 'folate', 'folic_acid', 'niacin',
       'pantothenic_acid', 'riboflavin', 'thiamin', 'vitamin_a',
       'vitamin_a_rae', 'carotene_alpha', 'carotene_beta',
       'cryptoxanthin_beta', 'lutein_zeaxanthin', 'vitamin_b12',
       'vitamin_b6', 'vitamin_c', 'vitamin_d', 'vitamin_e',
       'tocopherol_alpha', 'vitamin_k', 'calcium', 'copper', 'irom',
       'magnesium', 'manganese', 'phosphorous', 'potassium', 'selenium',
       'zink', 'protein', 'alanine', 'arginine', 'aspartic_acid',
       'cystine', 'glutamic_acid', 'glycine', 'histidine',
       'hydroxyproline', 'isoleucine', 'leucine', 'lysine', 'methionine',
       'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan',
       'tyrosine', 'valine', 'carbohydrate', 'fiber', 'sugars',
       'fructose', 'galactose', 'glucose', 'lactose', 'maltose',
       'sucrose', 'fat', 'saturated_fatty_acids',
       'monounsaturated_fatty

In [52]:
(water_df.drop(['name'], axis = 'columns').columns)

Index(['serving_size', 'total_fat', 'saturated_fat', 'cholesterol', 'sodium',
       'choline', 'folate', 'folic_acid', 'niacin', 'pantothenic_acid',
       'riboflavin', 'thiamin', 'vitamin_a', 'vitamin_a_rae', 'carotene_alpha',
       'carotene_beta', 'cryptoxanthin_beta', 'lutein_zeaxanthin',
       'vitamin_b12', 'vitamin_b6', 'vitamin_c', 'vitamin_d', 'vitamin_e',
       'tocopherol_alpha', 'vitamin_k', 'calcium', 'copper', 'irom',
       'magnesium', 'manganese', 'phosphorous', 'potassium', 'selenium',
       'zink', 'protein', 'alanine', 'arginine', 'aspartic_acid', 'cystine',
       'glutamic_acid', 'glycine', 'histidine', 'hydroxyproline', 'isoleucine',
       'leucine', 'lysine', 'methionine', 'phenylalanine', 'proline', 'serine',
       'threonine', 'tryptophan', 'tyrosine', 'valine', 'carbohydrate',
       'fiber', 'sugars', 'fructose', 'galactose', 'glucose', 'lactose',
       'maltose', 'sucrose', 'fat', 'saturated_fatty_acids',
       'monounsaturated_fatty_acids', 'poly

In [57]:
pd.DataFrame(scaler_water_mcar.transform(water_df[water_df.water_mcar.isnull()].drop(['name','water_mcar', 'water_mar'], axis = 'columns')),columns = water_df.drop(['name','water_mcar', 'water_mar'], axis = 'columns').columns)
# a = np.array(x) # your x
# b = np.array(y) # your y
# mses = ((a-b)**2).mean(axis=1)

C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- water
Feature names seen at fit time, yet now missing:
- water_mcar

  warnings.warn(message, FutureWarning)


,serving_size,total_fat,saturated_fat,cholesterol,sodium,choline,folate,folic_acid,niacin,pantothenic_acid,riboflavin,thiamin,vitamin_a,vitamin_a_rae,carotene_alpha,carotene_beta,cryptoxanthin_beta,lutein_zeaxanthin,vitamin_b12,vitamin_b6,vitamin_c,vitamin_d,vitamin_e,tocopherol_alpha,vitamin_k,calcium,copper,irom,magnesium,manganese,phosphorous,potassium,selenium,zink,protein,alanine,arginine,aspartic_acid,cystine,glutamic_acid,glycine,histidine,hydroxyproline,isoleucine,leucine,lysine,methionine,phenylalanine,proline,serine,threonine,tryptophan,tyrosine,valine,carbohydrate,fiber,sugars,fructose,galactose,glucose,lactose,maltose,sucrose,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water
0,0.0,-0.619342,-0.551556,-0.366113,-0.419401,-0.264566,-0.377104,-0.198124,-0.121463,0.295053,0.016684,0.315431,-0.165265,-0.119859,-0.06443,-0.101149,-0.039304,-0.039796,-0.312098,0.387865,-0.120460,-0.125774,-0.224649,-0.224649,-0.106650,0.565155,1.057292,0.776284,2.507023,1.367768,1.442629,0.469791,-0.355989,0.378058,0.005471,0.247723,-0.195034,-0.083019,0.785573,0.862210,-0.076787,-0.062212,-0.367017,0.059323,0.257674,-0.452719,0.554252,0.543382,0.213058,0.379865,0.157415,0.193964,0.252622,0.283687,1.776759,1.304084,-0.352894,0.311358,-0.062457,0.372412,-0.07869,-0.150615,0.023213,-0.620268,-0.544198,-0.549419,-0.273112,-0.366113,-0.024432,0.265352,-0.032626,-0.083373,-1.362212
1,0.0,-0.643570,-0.435218,-0.358304,-0.373641,-0.361717,-0.346635,-0.198124,-0.838286,-0.244720,-0.411286,-0.437932,-0.154655,-0.105630,-0.06443,-0.105163,0.004479,-0.109427,-0.283870,-0.564302,-0.070570,-0.125774,-0.241400,-0.241400,-0.137364,-0.136664,-0.267279,-0.475648,-0.467465,-0.075670,-0.778567,-0.580093,-0.455583,-0.510558,-1.183777,-0.859682,-0.839409,-0.841020,-0.883861,-0.933870,-0.779530,-0.854175,-0.367017,-0.891171,-0.904661,-0.834197,-0.856819,-0.915290,-0.882456,-0.907343,-0.882938,-0.868764,-0.882973,-0.905697,0.257549,-0.189534,1.322267,-0.162020,-0.062457,-0.142311,-0.07869,-0.173220,-0.156322,-0.643289,-0.440772,-0.557515,-0.454119,-0.358304,-0.024432,-0.642443,-0.032626,-0.083373,0.567750
2,0.0,-0.746538,-0.595182,-0.366113,-0.395175,0.296750,0.057075,-0.198124,-0.741841,0.088316,-0.502818,-0.390198,-0.167846,-0.119859,-0.06443,-0.106167,-0.039304,-0.116508,-0.312098,-0.230317,0.925066,-0.125774,-0.224649,-0.224649,0.113204,-0.314904,-0.248647,-0.428847,-0.349161,-0.053148,-0.755727,0.063794,-0.486491,-0.569799,-1.103844,-0.687716,-0.732221,-0.677402,-0.742383,-0.796040,-0.674929,-0.706833,-0.367017,-0.756470,-0.789298,-0.614036,-0.790881,-0.779454,-0.765315,-0.729369,-0.727905,-0.715853,-0.756520,-0.688973,-0.646582,-0.033484,-0.347678,0.814953,-0.062457,0.520483,-0.07869,-0.173220,-0.156322,-0.747491,-0.590602,-0.625578,-0.463069,-0.366113,-0.024432,-0.476551,-0.032626,-0.083373,1.442770
3,0.0,-0.710196,-0.580640,-0.366113,-0.388446,-0.500247,-0.232377,-0.198124,-0.495732,-0.325912,-0.282648,0.292602,-0.167846,-0.119859,-0.06443,-0.106167,-0.039304,-0.117689,-0.312098,-0.456431,-0.120460,-0.125774,-0.243793,-0.243793,-0.137364,-0.270344,2.032935,-0.164755,-0.197057,0.005816,-0.750017,0.974116,-0.483057,-0.253846,-0.160244,-0.271142,-0.325903,0.762798,0.509692,-0.504827,-0.204961,-0.220078,-0.367017,-0.084863,-0.208130,-0.233572,-0.385363,0.027205,0.237806,0.052889,0.037060,1.057909,-0.007722,0.132847,0.175776,-0.479340,-0.490007,-0.162020,-0.062457,-0.142311,-0.07869,-0.173220,-0.156322,-0.709930,-0.577219,-0.598133,-0.436950,-0.366113,-0.024432,0.094852,-0.032626,-0.083373,0.274617
4,0.0,-0.740481,-0.595182,-0.366113,-0.426130,-0.500247,-0.377104,-0.198124,-0.851971,-0.413117,-0.651247,-0.493967,-0.167846,-0.119859,-0.06443,-0.106167,-0.039304,-0.117689,-0.312098,-0.612014,-0.120460,-0.125774,-0.243793,-0.243793,-0.137364,-0.326044,-0.314706,-0.499049,-0.602669,-0.077391,-1.006968,-0.884591,-0.489926,-0.645966,-1.291004,-0.859682,-0.839409,-0.84102

In [41]:
scaler_water_mcar.transformwater_df[water_df.water_mcar.isnull()]['water']

1        8.82
2       66.10
3       92.07
13      57.40
20       3.50
        ...  
7166    62.22
7167    44.92
7170    54.74
7186    66.27
7187    47.33
Name: water, Length: 1476, dtype: float64

In [20]:
water_mcar_train_df = water_mcar_scaled_df[water_mcar_scaled_df.dataset_type == 'training'].drop(['name','dataset_type','serving_size'], axis = 'columns') 
water_mcar_test_df = water_mcar_scaled_df[water_mcar_scaled_df.dataset_type == 'validation'].drop(['name','dataset_type','serving_size'], axis = 'columns') 

water_mcar_train_df

,total_fat,saturated_fat,cholesterol,sodium,choline,folate,folic_acid,niacin,pantothenic_acid,riboflavin,thiamin,vitamin_a,vitamin_a_rae,carotene_alpha,carotene_beta,cryptoxanthin_beta,lutein_zeaxanthin,vitamin_b12,vitamin_b6,vitamin_c,vitamin_d,vitamin_e,tocopherol_alpha,vitamin_k,calcium,copper,irom,magnesium,manganese,phosphorous,potassium,selenium,zink,protein,alanine,arginine,aspartic_acid,cystine,glutamic_acid,glycine,histidine,hydroxyproline,isoleucine,leucine,lysine,methionine,phenylalanine,proline,serine,threonine,tryptophan,tyrosine,valine,carbohydrate,fiber,sugars,fructose,galactose,glucose,lactose,maltose,sucrose,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water_mcar
0,3.596307,0.291893,-0.366113,-0.435551,0.228384,-0.209526,-0.198124,-0.598476,0.235663,-0.329650,0.875784,-0.151787,-0.116302,-0.06443,-0.077065,0.039505,-0.097625,-0.312098,-0.176382,-0.096600,-0.125774,0.091222,0.091222,-0.080784,-0.047544,1.717884,-0.076167,1.442291,0.626420,0.574706,0.415869,-0.376594,0.631948,-0.397119,-0.271142,0.627573,0.017739,0.191368,0.047029,-0.112145,-0.164825,-0.367017,-0.253714,-0.253840,-0.543016,-0.253487,-0.025040,-0.283553,0.073584,-0.258726,-0.157730,-0.349888,-0.193107,-0.330509,1.660769,-0.194171,-0.121733,-0.062457,-0.114107,-0.07869,-0.17322,0.973011,3.595641,0.289465,4.968632,3.479097,-0.366113,-0.024432,-0.140161,-0.032626,-0.083373,-1.540787
4,-0.722310,-0.580640,-0.366113,-0.431514,-0.269964,0.582661,-0.198124,-0.523319,-0.349968,0.476815,-0.060213,1.215804,0.165906,-0.06443,2.799010,-0.039304,2.162443,-0.312098,-0.309146,1.007493,-0.125774,0.239586,0.239586,1.618233,0.158546,0.142626,-0.122968,0.157854,0.034281,-0.664367,1.170771,-0.476189,-0.530305,-0.805557,-0.859682,-0.565207,-0.841020,-0.431133,-0.933870,-0.779530,-0.554229,-0.367017,-0.397900,-0.478036,-0.584613,-0.596364,-0.507781,-0.882456,-0.907343,-0.542273,-0.501779,-0.441628,-0.461845,-0.585074,0.345494,-0.265708,-0.162020,-0.062457,-0.142311,-0.07869,-0.17322,-0.156322,-0.719623,-0.587547,-0.622010,-0.412658,-0.366113,-0.024432,0.057987,-0.032626,-0.083373,1.226795
5,0.628393,0.844497,0.203946,-0.356145,0.746522,-0.239995,-0.198124,0.442654,0.075535,-0.131745,-0.265675,-0.167846,-0.119859,-0.06443,-0.106167,-0.039304,-0.117689,0.189493,-0.342337,-0.120460,-0.110899,-0.195934,-0.195934,-0.079168,-0.348324,-0.143630,-0.239971,-0.247758,-0.074419,-0.110495,-0.180439,0.138548,0.315996,0.323254,0.616858,0.387023,0.505818,0.516766,0.354331,0.412333,0.524525,-0.367017,0.624687,0.497107,0.649102,0.544361,0.493228,0.262554,0.365378,0.563356,0.606823,0.495610,0.642583,-0.823283,-0.479340,-0.490007,-0.162020,-0.062457,-0.142311,-0.07869,-0.17322,-0.156322,0.653774,0.872782,0.687107,-0.130826,0.203946,-0.024432,-0.425862,-0.032626,-0.083373,0.344363
6,0.688963,1.571608,0.196137,0.697674,-0.223187,0.095161,-0.198124,-0.715123,0.612302,0.555977,-0.435857,0.067303,0.165906,-0.06443,-0.094124,-0.039304,-0.117689,-0.029817,-0.141117,-0.120460,0.008100,-0.193541,-0.193541,-0.105033,1.723714,-0.279136,-0.443890,-0.264659,-0.071447,0.974408,-0.291454,-0.009126,0.025433,0.639086,0.354461,0.034299,0.349596,-0.112809,1.311633,-0.221166,0.942871,-0.367017,0.945312,1.097864,0.957531,1.005926,1.393926,2.988139,1.398041,0.579676,1.478413,1.956015,1.311829,-0.806929,-0.479340,-0.455729,-0.162020,-0.062457,-0.142311,-0.07869,-0.17322,-0.156322,0.705269,1.610147,0.333480,-0.336492,0.196137,-0.024432,0.869012,-0.032626,-0.083373,0.085934
7,0.325545,-0.202543,-0.366113,0.223927,0.975007,0.399848,-0.198124,1.754657,-0.413117,1.575191,1.788951,-0.167846,-0.119859,-0.06443,-0.106167,-0.039304,-0.117689,0.599886,2.499641,-0.120460,-0.125774,0.581780,0.581780,-0.137364,0.091706,1.252082,-0.164755,-0.213957,-0.077391,1.562539,1.018522,-0.472754,-0.251025,0.951020,-0.859682,-0.839409,-0.841020,-0.883861,-0.933870,-0.779530,-0.854175,-0.367017,-0.891171,-0.904661,-0.834197,-0.856819,-0.91529

In [32]:
input_shape = 72
latent_shape = 40
enc_input = keras.Input(shape = (input_shape,),name = 'input_data')
latent_layer = keras.layers.Dense(latent_shape, activation = 'relu')(enc_input)
encoder = keras.Model(enc_input,latent_layer, name = "encoder")

# dec_output = keras.layers.Dense(input_shape, activation = 'relu')(latent_layer)
dec_output = keras.layers.Dense(input_shape)(latent_layer)
# dec_output_reshaped = keras.layers.Reshape((input_shape,))(dec_output)
opt = keras.optimizers.Adam(lr = 0.0001)
auto_encoder = keras.Model(enc_input,dec_output, name="auto_encoder")
auto_encoder.summary()

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_data (InputLayer)     [(None, 72)]              0         
                                                                 
 dense_2 (Dense)             (None, 40)                2920      
                                                                 
 dense_3 (Dense)             (None, 72)                2952      
                                                                 
Total params: 5,872
Trainable params: 5,872
Non-trainable params: 0
_________________________________________________________________


C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [33]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
auto_encoder.compile(opt, loss = "mse")
auto_encoder.fit(water_mcar_train_df,water_mcar_train_df, epochs = 1000, validation_split = 0.1, callbacks=[callback])

Epoch 1/1000
161/161 [==============================] - 2s 9ms/step - loss: 1.3920 - val_loss: 0.7828
Epoch 2/1000
161/161 [==============================] - 1s 7ms/step - loss: 1.1802 - val_loss: 0.5791
Epoch 3/1000
161/161 [==============================] - 1s 6ms/step - loss: 1.0467 - val_loss: 0.4463
Epoch 4/1000
161/161 [==============================] - 1s 6ms/step - loss: 0.9464 - val_loss: 0.3464
Epoch 5/1000
161/161 [==============================] - 1s 7ms/step - loss: 0.8681 - val_loss: 0.2805
Epoch 6/1000
161/161 [==============================] - 2s 10ms/step - loss: 0.8067 - val_loss: 0.2419
Epoch 7/1000
161/161 [==============================] - 2s 9ms/step - loss: 0.7585 - val_loss: 0.2192
Epoch 8/1000
161/161 [==============================] - 2s 10ms/step - loss: 0.7188 - val_loss: 0.2057
Epoch 9/1000
161/161 [==============================] - 2s 10ms/step - loss: 0.6845 - val_loss: 0.1973
Epoch 10/1000
161/161 [==============================] - 2s 10ms/step - loss: 0

161/161 [==============================] - 1s 6ms/step - loss: 0.1335 - val_loss: 0.0696
Epoch 81/1000
161/161 [==============================] - 1s 7ms/step - loss: 0.1315 - val_loss: 0.0690
Epoch 82/1000
161/161 [==============================] - 1s 8ms/step - loss: 0.1296 - val_loss: 0.0685
Epoch 83/1000
161/161 [==============================] - 1s 6ms/step - loss: 0.1276 - val_loss: 0.0676
Epoch 84/1000
161/161 [==============================] - 1s 6ms/step - loss: 0.1256 - val_loss: 0.0675
Epoch 85/1000
161/161 [==============================] - 1s 6ms/step - loss: 0.1238 - val_loss: 0.0669
Epoch 86/1000
161/161 [==============================] - 1s 7ms/step - loss: 0.1219 - val_loss: 0.0667
Epoch 87/1000
161/161 [==============================] - 1s 6ms/step - loss: 0.1200 - val_loss: 0.0665
Epoch 88/1000
161/161 [==============================] - 1s 6ms/step - loss: 0.1182 - val_loss: 0.0660
Epoch 89/1000
161/161 [==============================] - 2s 11ms/step - loss: 0.1164 - 

161/161 [==============================] - 1s 6ms/step - loss: 0.0491 - val_loss: 0.0458
Epoch 160/1000
161/161 [==============================] - 1s 6ms/step - loss: 0.0487 - val_loss: 0.0458
Epoch 161/1000
161/161 [==============================] - 2s 10ms/step - loss: 0.0483 - val_loss: 0.0454
Epoch 162/1000
161/161 [==============================] - 2s 9ms/step - loss: 0.0480 - val_loss: 0.0454
Epoch 163/1000
161/161 [==============================] - 1s 6ms/step - loss: 0.0475 - val_loss: 0.0452
Epoch 164/1000
161/161 [==============================] - 1s 6ms/step - loss: 0.0472 - val_loss: 0.0450
Epoch 165/1000
161/161 [==============================] - 2s 11ms/step - loss: 0.0468 - val_loss: 0.0450
Epoch 166/1000
161/161 [==============================] - 2s 10ms/step - loss: 0.0465 - val_loss: 0.0445
Epoch 167/1000
161/161 [==============================] - 1s 6ms/step - loss: 0.0461 - val_loss: 0.0444
Epoch 168/1000
161/161 [==============================] - 1s 6ms/step - loss

In [16]:
water_mcar_train_df[water_mcar_train_df.index == 0] 

,total_fat,saturated_fat,cholesterol,sodium,choline,folate,folic_acid,niacin,pantothenic_acid,riboflavin,thiamin,vitamin_a,vitamin_a_rae,carotene_alpha,carotene_beta,cryptoxanthin_beta,lutein_zeaxanthin,vitamin_b12,vitamin_b6,vitamin_c,vitamin_d,vitamin_e,tocopherol_alpha,vitamin_k,calcium,copper,irom,magnesium,manganese,phosphorous,potassium,selenium,zink,protein,alanine,arginine,aspartic_acid,cystine,glutamic_acid,glycine,histidine,hydroxyproline,isoleucine,leucine,lysine,methionine,phenylalanine,proline,serine,threonine,tryptophan,tyrosine,valine,carbohydrate,fiber,sugars,fructose,galactose,glucose,lactose,maltose,sucrose,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water_mcar
0,3.596307,0.291893,-0.366113,-0.435551,0.228384,-0.209526,-0.198124,-0.598476,0.235663,-0.32965,0.875784,-0.151787,-0.116302,-0.06443,-0.077065,0.039505,-0.097625,-0.312098,-0.176382,-0.0966,-0.125774,0.091222,0.091222,-0.080784,-0.047544,1.717884,-0.076167,1.442291,0.62642,0.574706,0.415869,-0.376594,0.631948,-0.397119,-0.271142,0.627573,0.017739,0.191368,0.047029,-0.112145,-0.164825,-0.367017,-0.253714,-0.25384,-0.543016,-0.253487,-0.02504,-0.283553,0.073584,-0.258726,-0.15773,-0.349888,-0.193107,-0.330509,1.660769,-0.194171,-0.121733,-0.062457,-0.114107,-0.07869,-0.17322,0.973011,3.595641,0.289465,4.968632,3.479097,-0.366113,-0.024432,-0.140161,-0.032626,-0.083373,-1.540787


In [34]:
max_water_mcar = water_mcar_train_df.water_mcar.max()
min_water_mcar = water_mcar_train_df.water_mcar.min()

In [67]:
water_mcar_test_df['water_mcar'] = 0#np.random.uniform(min_water_mcar, max_water_mcar, size=len(water_mcar_test_df))

In [37]:
water_mcar_test_df

,total_fat,saturated_fat,cholesterol,sodium,choline,folate,folic_acid,niacin,pantothenic_acid,riboflavin,thiamin,vitamin_a,vitamin_a_rae,carotene_alpha,carotene_beta,cryptoxanthin_beta,lutein_zeaxanthin,vitamin_b12,vitamin_b6,vitamin_c,vitamin_d,vitamin_e,tocopherol_alpha,vitamin_k,calcium,copper,irom,magnesium,manganese,phosphorous,potassium,selenium,zink,protein,alanine,arginine,aspartic_acid,cystine,glutamic_acid,glycine,histidine,hydroxyproline,isoleucine,leucine,lysine,methionine,phenylalanine,proline,serine,threonine,tryptophan,tyrosine,valine,carbohydrate,fiber,sugars,fructose,galactose,glucose,lactose,maltose,sucrose,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water_mcar
1,-0.619342,-0.551556,-0.366113,-0.419401,-0.264566,-0.377104,-0.198124,-0.121463,0.295053,0.016684,0.315431,-0.165265,-0.119859,-0.06443,-0.101149,-0.039304,-0.039796,-0.312098,0.387865,-0.120460,-0.125774,-0.224649,-0.224649,-0.106650,0.565155,1.057292,0.776284,2.507023,1.367768,1.442629,0.469791,-0.355989,0.378058,0.005471,0.247723,-0.195034,-0.083019,0.785573,0.862210,-0.076787,-0.062212,-0.367017,0.059323,0.257674,-0.452719,0.554252,0.543382,0.213058,0.379865,0.157415,0.193964,0.252622,0.283687,1.776759,1.304084,-0.352894,0.311358,-0.062457,0.372412,-0.07869,-0.150615,0.023213,-0.620268,-0.544198,-0.549419,-0.273112,-0.366113,-0.024432,0.265352,-0.032626,-0.083373,-0.003348
2,-0.643570,-0.435218,-0.358304,-0.373641,-0.361717,-0.346635,-0.198124,-0.838286,-0.244720,-0.411286,-0.437932,-0.154655,-0.105630,-0.06443,-0.105163,0.004479,-0.109427,-0.283870,-0.564302,-0.070570,-0.125774,-0.241400,-0.241400,-0.137364,-0.136664,-0.267279,-0.475648,-0.467465,-0.075670,-0.778567,-0.580093,-0.455583,-0.510558,-1.183777,-0.859682,-0.839409,-0.841020,-0.883861,-0.933870,-0.779530,-0.854175,-0.367017,-0.891171,-0.904661,-0.834197,-0.856819,-0.915290,-0.882456,-0.907343,-0.882938,-0.868764,-0.882973,-0.905697,0.257549,-0.189534,1.322267,-0.162020,-0.062457,-0.142311,-0.07869,-0.173220,-0.156322,-0.643289,-0.440772,-0.557515,-0.454119,-0.358304,-0.024432,-0.642443,-0.032626,-0.083373,1.408102
3,-0.746538,-0.595182,-0.366113,-0.395175,0.296750,0.057075,-0.198124,-0.741841,0.088316,-0.502818,-0.390198,-0.167846,-0.119859,-0.06443,-0.106167,-0.039304,-0.116508,-0.312098,-0.230317,0.925066,-0.125774,-0.224649,-0.224649,0.113204,-0.314904,-0.248647,-0.428847,-0.349161,-0.053148,-0.755727,0.063794,-0.486491,-0.569799,-1.103844,-0.687716,-0.732221,-0.677402,-0.742383,-0.796040,-0.674929,-0.706833,-0.367017,-0.756470,-0.789298,-0.614036,-0.790881,-0.779454,-0.765315,-0.729369,-0.727905,-0.715853,-0.756520,-0.688973,-0.646582,-0.033484,-0.347678,0.814953,-0.062457,0.520483,-0.07869,-0.173220,-0.156322,-0.747491,-0.590602,-0.625578,-0.463069,-0.366113,-0.024432,-0.476551,-0.032626,-0.083373,1.651227
13,-0.710196,-0.580640,-0.366113,-0.388446,-0.500247,-0.232377,-0.198124,-0.495732,-0.325912,-0.282648,0.292602,-0.167846,-0.119859,-0.06443,-0.106167,-0.039304,-0.117689,-0.312098,-0.456431,-0.120460,-0.125774,-0.243793,-0.243793,-0.137364,-0.270344,2.032935,-0.164755,-0.197057,0.005816,-0.750017,0.974116,-0.483057,-0.253846,-0.160244,-0.271142,-0.325903,0.762798,0.509692,-0.504827,-0.204961,-0.220078,-0.367017,-0.084863,-0.208130,-0.233572,-0.385363,0.027205,0.237806,0.052889,0.037060,1.057909,-0.007722,0.132847,0.175776,-0.479340,-0.490007,-0.162020,-0.062457,-0.142311,-0.07869,-0.173220,-0.156322,-0.709930,-0.577219,-0.598133,-0.436950,-0.366113,-0.024432,0.094852,-0.032626,-0.083373,-1.192960
20,-0.740481,-0.595182,-0.366113,-0.426130,-0.500247,-0.377104,-0.198124,-0.851971,-0.413117,-0.651247,-0.493967,-0.167846,-0.119859,-0.06443,-0.106167,-0.039304,-0.117689,-0.312098,-0.612014,-0.120460,-0.125774,-0.243793,-0.243793,-0.137364,-0.326044,-0.314706,-0.499049,-0.602669,-0.077391,-1.006968,-0.884591,-0.489926,-0.645966,-1.291004,-0.859682,-0.839409,-0.841020,-0.883861,-0.933870,-0.

In [58]:
water_mcar_test_df

,total_fat,saturated_fat,cholesterol,sodium,choline,folate,folic_acid,niacin,pantothenic_acid,riboflavin,thiamin,vitamin_a,vitamin_a_rae,carotene_alpha,carotene_beta,cryptoxanthin_beta,lutein_zeaxanthin,vitamin_b12,vitamin_b6,vitamin_c,vitamin_d,vitamin_e,tocopherol_alpha,vitamin_k,calcium,copper,irom,magnesium,manganese,phosphorous,potassium,selenium,zink,protein,alanine,arginine,aspartic_acid,cystine,glutamic_acid,glycine,histidine,hydroxyproline,isoleucine,leucine,lysine,methionine,phenylalanine,proline,serine,threonine,tryptophan,tyrosine,valine,carbohydrate,fiber,sugars,fructose,galactose,glucose,lactose,maltose,sucrose,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water_mcar
1,-0.619342,-0.551556,-0.366113,-0.419401,-0.264566,-0.377104,-0.198124,-0.121463,0.295053,0.016684,0.315431,-0.165265,-0.119859,-0.06443,-0.101149,-0.039304,-0.039796,-0.312098,0.387865,-0.120460,-0.125774,-0.224649,-0.224649,-0.106650,0.565155,1.057292,0.776284,2.507023,1.367768,1.442629,0.469791,-0.355989,0.378058,0.005471,0.247723,-0.195034,-0.083019,0.785573,0.862210,-0.076787,-0.062212,-0.367017,0.059323,0.257674,-0.452719,0.554252,0.543382,0.213058,0.379865,0.157415,0.193964,0.252622,0.283687,1.776759,1.304084,-0.352894,0.311358,-0.062457,0.372412,-0.07869,-0.150615,0.023213,-0.620268,-0.544198,-0.549419,-0.273112,-0.366113,-0.024432,0.265352,-0.032626,-0.083373,-0.003348
2,-0.643570,-0.435218,-0.358304,-0.373641,-0.361717,-0.346635,-0.198124,-0.838286,-0.244720,-0.411286,-0.437932,-0.154655,-0.105630,-0.06443,-0.105163,0.004479,-0.109427,-0.283870,-0.564302,-0.070570,-0.125774,-0.241400,-0.241400,-0.137364,-0.136664,-0.267279,-0.475648,-0.467465,-0.075670,-0.778567,-0.580093,-0.455583,-0.510558,-1.183777,-0.859682,-0.839409,-0.841020,-0.883861,-0.933870,-0.779530,-0.854175,-0.367017,-0.891171,-0.904661,-0.834197,-0.856819,-0.915290,-0.882456,-0.907343,-0.882938,-0.868764,-0.882973,-0.905697,0.257549,-0.189534,1.322267,-0.162020,-0.062457,-0.142311,-0.07869,-0.173220,-0.156322,-0.643289,-0.440772,-0.557515,-0.454119,-0.358304,-0.024432,-0.642443,-0.032626,-0.083373,1.408102
3,-0.746538,-0.595182,-0.366113,-0.395175,0.296750,0.057075,-0.198124,-0.741841,0.088316,-0.502818,-0.390198,-0.167846,-0.119859,-0.06443,-0.106167,-0.039304,-0.116508,-0.312098,-0.230317,0.925066,-0.125774,-0.224649,-0.224649,0.113204,-0.314904,-0.248647,-0.428847,-0.349161,-0.053148,-0.755727,0.063794,-0.486491,-0.569799,-1.103844,-0.687716,-0.732221,-0.677402,-0.742383,-0.796040,-0.674929,-0.706833,-0.367017,-0.756470,-0.789298,-0.614036,-0.790881,-0.779454,-0.765315,-0.729369,-0.727905,-0.715853,-0.756520,-0.688973,-0.646582,-0.033484,-0.347678,0.814953,-0.062457,0.520483,-0.07869,-0.173220,-0.156322,-0.747491,-0.590602,-0.625578,-0.463069,-0.366113,-0.024432,-0.476551,-0.032626,-0.083373,1.651227
13,-0.710196,-0.580640,-0.366113,-0.388446,-0.500247,-0.232377,-0.198124,-0.495732,-0.325912,-0.282648,0.292602,-0.167846,-0.119859,-0.06443,-0.106167,-0.039304,-0.117689,-0.312098,-0.456431,-0.120460,-0.125774,-0.243793,-0.243793,-0.137364,-0.270344,2.032935,-0.164755,-0.197057,0.005816,-0.750017,0.974116,-0.483057,-0.253846,-0.160244,-0.271142,-0.325903,0.762798,0.509692,-0.504827,-0.204961,-0.220078,-0.367017,-0.084863,-0.208130,-0.233572,-0.385363,0.027205,0.237806,0.052889,0.037060,1.057909,-0.007722,0.132847,0.175776,-0.479340,-0.490007,-0.162020,-0.062457,-0.142311,-0.07869,-0.173220,-0.156322,-0.709930,-0.577219,-0.598133,-0.436950,-0.366113,-0.024432,0.094852,-0.032626,-0.083373,-1.192960
20,-0.740481,-0.595182,-0.366113,-0.426130,-0.500247,-0.377104,-0.198124,-0.851971,-0.413117,-0.651247,-0.493967,-0.167846,-0.119859,-0.06443,-0.106167,-0.039304,-0.117689,-0.312098,-0.612014,-0.120460,-0.125774,-0.243793,-0.243793,-0.137364,-0.326044,-0.314706,-0.499049,-0.602669,-0.077391,-1.006968,-0.884591,-0.489926,-0.645966,-1.291004,-0.859682,-0.839409,-0.841020,-0.883861,-0.933870,-0.

In [69]:
water_mcar_pred = pd.DataFrame(auto_encoder.predict(water_mcar_test_df), columns = water_mcar_test_df.columns)

47/47 [==============================] - 0s 1ms/step


In [71]:
water_mcar_true = pd.DataFrame(scaler_water_mcar.transform(water_df[water_df.water_mcar.isnull()].drop(['name','water_mcar', 'water_mar'], axis = 'columns')),columns = water_df.drop(['name','water_mcar', 'water_mar'], axis = 'columns').columns).drop('serving_size',axis = 'columns')# a = np.array(x) # your x
# b = np.array(y) # your y
# mses = ((a-b)**2).mean(axis=1)

C:\Users\Chike\Anaconda3\envs\imputation_research\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- water
Feature names seen at fit time, yet now missing:
- water_mcar

  warnings.warn(message, FutureWarning)


In [77]:
from sklearn.metrics import mean_squared_error
pd.DataFrame({'columns':water_mcar_true.columns,
              'mse':list(mean_squared_error(water_mcar_true, water_mcar_pred, multioutput='raw_values'))}).nlargest(10,'mse')

,columns,mse
31,selenium,1.593192
57,galactose,0.668080
58,glucose,0.558205
29,phosphorous,0.404909
6,folic_acid,0.256179
56,fructose,0.250337
71,water,0.208576
5,folate,0.201098
33,protein,0.189536
17,vitamin_b12,0.161144
